In [2]:
import pandas as pd

In [3]:
df = pd.read_csv("spotify_millsongdata.csv")

In [4]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [5]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [6]:
df.shape

(57650, 4)

In [7]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [8]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [9]:
df.head(10)

,artist,song,text
0,"Harry Connick, Jr.",Winter Wonderland,Sleigh bells ring \r\nAre you listening \r\n...
1,Bryan White,On Any Given Night,"I'd be foolin' myself, thinkin' I've got it ma..."
2,Proclaimers,When Love Struck You Down,So you're back on top again \r\nProving every...
3,Robbie Williams,Eternity,Close your eyes so your don't feel them \r\nT...
4,Deep Purple,Soldier Of Fortune,I have often told you stories \r\nAbout the w...
5,Scorpions,Arizona,Arizona really was a gas \r\nI was screwed up...
6,Used,Put Me Out,You bought your smile on a corner \r\nYou pai...
7,Guns N' Roses,Shadow Of Your Love,Took my chances carelessly \r\nchoked on my o...
8,Lil Wayne,Coco,"[Intro] \r\nI'm sorry for the wait, I'm sorry..."
9,Indigo Girls,Pendulum Swinger,I meet you for coffee \r\nWe get together per...


In [10]:
df['text'][0]

"Sleigh bells ring  \r\nAre you listening  \r\nIn the lane  \r\nSnow is glistening  \r\nA beautiful sight  \r\nWe're happy tonight  \r\nWalking in a winter wonderland  \r\n  \r\nGone away is the bluebird  \r\nHere to stay is a new bird  \r\nHe sings a love song  \r\nAs we go along  \r\nWalking in a winter wonderland  \r\n  \r\nIn the meadow we can build a snowman  \r\nThen pretend that he is Parson Brown  \r\nHe'll say: Are you married?  \r\nWe'll say: No man  \r\nBut you can do the job  \r\nWhen you're in town  \r\n  \r\nLater on  \r\nWe'll conspire  \r\nAs we dream by the fire  \r\nTo face unafraid  \r\nThe plans that we've made  \r\nWalking in a winter wonderland  \r\n  \r\nIn the meadow we can build a snowman  \r\nAnd pretend that he's a circus clown  \r\nWe'll have lots of fun with mister snowman  \r\nUntil the all the kids knock him down  \r\n  \r\nWhen it snows  \r\nAin't it thrilling  \r\nThough your nose gets a chilling  \r\nWe'll frolic and play  \r\nThe Eskimo way  \r\nWalki

In [11]:
# df = df.sample(5000)

In [12]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [13]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [14]:
#pip install nltk

In [15]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [16]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [17]:
# pip install scikit-learn

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [24]:
similarity[0]

array([1.        , 0.04641796, 0.00701178, ..., 0.00860774, 0.01322995,
       0.03761798])

In [25]:
df[df['song'] == 'When Love Struck You Down']

,artist,song,text
2,Proclaimers,When Love Struck You Down,so you 're back on top again prove everybodi w...


In [26]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [27]:
recommendation('When Love Struck You Down')

['Fool For Your Love',
 'Love You So',
 'When We Die',
 'Love Is The Only Way',
 'Love Surrounds Me',
 'All You Need Is Love',
 'But You Know I Love You',
 'Is It Love',
 'For You To Love',
 'And I Love Her',
 'Cold Canyon',
 'No Love',
 'I Want Your Love',
 'As Long As You Love Me',
 'Love Is Gone',
 "Big Boys Don't Cry",
 "I'll Love You For A While",
 "Junior's Wailing",
 'Blame It On Mexico',
 'Accidentally In Love - Counting Crows']

In [28]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))